# modèles d'espace d'état

Un modèle d'espace d'état est un cadre mathématique qui permet de modéliser et prédire comment un système évolue au fil du temps. Dans cette partie, l'objectif est de modéliser les systèmes électroniques.

La clé pour comprendre les modèles d'espace d'état et que tous les systèmes ont des été internes qui ne sont pas visible et qui influence la sortie dy système. Tout l'objectif de ce système est d'apprendre sur ces état cachés, afin de voir comment il évolue dans le temps afin de faire des prédictions futures.

Les points clés sont les suivants :
* Les modèles vont séparer les sortiés qui sont observables des états qui sont caché.
* Il peuvent modéliser des systèmes qui évolue au cours du temps.
* Ils vont très bien supporté la bruit et les incertitudes de mesure.
* Les modèles linéaires utilisent des matrices pour décrires les transitions d'état et les observations

```{note} Définition
L’espace d’état est une représentation mathématique de l’état d’un système à un moment donné, définie par des variables d’état collectées dans un vecteur d’état, où chaque point représente un état système unique.
```

La [](#modele_espace_etat) donne une représentation de ce que l'on va étudier dans cette partie.

```{figure} ./images/modele_espace_etat.svg
:name: modele_espace_etat
:align: center
:width: 400px

Modèle d'espace d'état
```

Il est possible de voir qu'il y a plusieurs matrice représenté sur se diagramme.

* **input** ($x(t)$) : Le signal externe ou l'observation introduit dans le système à l'instant t.
* **Matrice** $B$ (*cartographie entrée-état*): Transforme l'entrée $x(t)$ dans l'espace d'état multiplié par $x(T)$ influencer la mise à jour de l'état.
* **Matrice** $A$ (*transition de l'état*): Représente la dynamique du système; multiplie l'état précédent pour produire l'état suivant prédit.
* **Représentation de l'État**: L'état caché ou latent qui évolue au fil du temps; mis à jour en utilisant les contributions de $A$ (*état précédent*) et B (*entrée courante*).
* **Matrice** $D$ (*cartographie entrée-sortie*): Directement les cartes d'entrée $x(T)$ à la sortie sans passer par l'état, permettant des effets d'entrée immédiats sur la sortie.
* **Matrice** $C$ (*cartographie d'état à sortie*): Convertit la représentation actuelle de l'état en contributions de sortie.
* **Opérations de somme**: Combiner les contributions des chemins pilotés par l'état ($C$ x état) et des voies d'entrée (entrée x $D$ ) pour former la sortie finale.
* **La sortie** $y(T)$: Le signal observé ou prédit au temps ttt, combinant les effets à la fois de l'état actuel et de l'entrée directe.
* **Mises à jour** de la représentation de l'état est mise à jour pendant l'entraînement pour minimiser l'erreur de prédiction, en apprenant efficacement la dynamique du système encodée en $A$, $B$, $C$, $D$.

In [2]:
import torch
from torch import Tensor
from torch.distributions import Normal
import matplotlib.pyplot as plt
import math

In [3]:
def next_obs(current_l: Tensor = torch.tensor([0., 0.]),
             a: Tensor = torch.tensor([1., 1.]),
             F: Tensor = torch.tensor([[1., 1.], [0., 1.]]),
             alpha: float = 0.6, beta: float = 0.6,
             sigma_t: float = 3.0):
 
    g = torch.tensor([alpha, beta])

    y_t = (a * current_l).sum()                      
    obs_noise = sigma_t * torch.randn(1).item()      
    z_t = y_t + obs_noise
    state_noise = torch.randn(1).item()

    next_l = F.matmul(current_l) + g * state_noise
    return next_l, z_t